<a href="https://colab.research.google.com/github/Carlageo9/Practica_3_Metricas/blob/main/Practica_3_Metricas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tópicos de Indistria I - Práctica 3 Métricas

**Nombre:** Carla Georgina Sánchez Arreguín

**e-mail:** carla.sanchez2472@alumnos.udg.mx





MODULES

In [2]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import cauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

CLASSES

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [4]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [5]:
# Define your function to compute path length for given trajectory
###########################################################
### Brownian Motion (BM)
###########################################################
def bm_2d(n_steps = 1000, speed = 6, s_pos = [0, 0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Returns:
    BM_2d_df:
  """

  #Inicializamos el vector de velocidad
  velocity = Vec2d(speed, 0)

  #Declaramos los DataFrame para guardar la trayectoria
  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  #con un for repetimos ciertas condiciones según el número de pasos definido
  for i in range(n_steps-1):
    #Generamos un ángulo de giro aleatorio
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    #Actualizamos la dirección en la que se generará el movimiento
    velocity = velocity.rotated(turn_angle)
    #Calculamos la nueva posición sumando la actual con recién generada
    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
    #Actualizamos la nueva posición
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df




In [6]:
# Define your function to compute path length for given trajectory
###########################################################
### Correlated Random Walks (CRW)
###########################################################
def crw_2d(n_steps = 1000, speed = 6, s_pos = [0, 0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Returns:
    CRW_2d_df:
  """

  #Inicializamos el vector de velocidad
  velocity = Vec2d(speed, 0)

  #Declaramos los DataFrame para guardar la trayectoria
  CRW_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  distribucion = wrapcauchy(0.4, 0, scale=1)
  giros = distribucion.rvs(size=n_steps)

  #con un for repetimos ciertas condiciones según el número de pasos definido
  for i in range(n_steps-1):
    #Generamos un ángulo de giro aleatorio
    turn_angle = giros[i]
    #Actualizamos la dirección en la que se generará el movimiento
    velocity = velocity.rotated(turn_angle)
    #Calculamos la nueva posición sumando la actual con recién generada
    temp_df = pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i]+velocity.x, 'y_pos': CRW_2d_df.y_pos[i]+velocity.y}])
    #Actualizamos la nueva posición
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  return CRW_2d_df




In [7]:
###########################################################
### Función para calcular distancia entre 2 puntos
###########################################################
def distancia(x1, y1, x2, y2):
    """
    Calcula la distancia euclidiana entre dos puntos en un espacio bidimensional.

    Args:
        x1 (float): Coordenada x del primer punto.
        y1 (float): Coordenada y del primer punto.
        x2 (float): Coordenada x del segundo punto.
        y2 (float): Coordenada y del segundo punto.

    Returns:
        float: Distancia euclidiana entre los dos puntos.
    """
    #Con la función math.sqrt calculamos la raíz cuadrada, y con ** elevamos a la segunda potencia
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

In [8]:
# Generamos trayectorias
BM_2d_df_3 = bm_2d(1000, 3, [0, 0])
BM_2d_df_6 = bm_2d(1000, 6, [0, 0])
#CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')
CRW_2d_df_9 = crw_2d(1000, 6, [0, 0])


#Calculamos distancia y path length para BM3
dis_BM_3 = np.array([distancia(BM_2d_df_3['x_pos'].iloc[i-1], BM_2d_df_3['y_pos'].iloc[i-1], BM_2d_df_3['x_pos'].iloc[i], BM_2d_df_3['y_pos'].iloc[i]) for i in range(1, BM_2d_df_3.shape[0])])
pl_BM_3 = np.cumsum(dis_BM_3)

#Calculamos distancia y path length para BM6
dis_BM_6 = np.array([distancia(BM_2d_df_6['x_pos'].iloc[i-1], BM_2d_df_6['y_pos'].iloc[i-1], BM_2d_df_6['x_pos'].iloc[i], BM_2d_df_6['y_pos'].iloc[i]) for i in range(1, BM_2d_df_6.shape[0])])
pl_BM_6 = np.cumsum(dis_BM_6)

#Calculamos distancia y path length para CRW6
dis_CRW_6 = np.array([distancia(CRW_2d_df_9['x_pos'].iloc[i-1], CRW_2d_df_9['y_pos'].iloc[i-1], CRW_2d_df_9['x_pos'].iloc[i], CRW_2d_df_9['y_pos'].iloc[i]) for i in range(1, CRW_2d_df_9.shape[0])])
pl_CRW_6 = np.cumsum(dis_CRW_6)

#Creamos un DataFrame para almacenar las métricas
df_metrics = pd.DataFrame({
    'Step': np.arange(2, BM_2d_df_3.shape[0] + 1),
    'Path Length BM 3': pl_BM_3,
    'Path Length BM 6': pl_BM_6,
    'Path Length CRW 6': pl_CRW_6
})

# Inicializamos figura
fig_path_length = go.Figure()

# Añadimos trayectoria BM3
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(pl_BM_3)) + 1,
    y=pl_BM_3,
    name='Path Length BM 3',
    showlegend=True))

# Añadimos trayectoria BM6
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(pl_BM_6)) + 1,
    y=pl_BM_6, line=dict(width=8),
    name='Path Length BM 6',
    showlegend=True))

# Añadimos trayectoria CRW6
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(pl_CRW_6)) + 1,
    y=pl_CRW_6,
    name='Path Length CRW 6',
    showlegend=True))

fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [9]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [10]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [123]:
def funcion_msd(trajectory):
    ##Número de pasos que conforma la trayectoria (brownian_6.csv)
    ##n_steps = 1000
    # Obtenemos el tamaño de la trayectoria con .shape[0] para que la función sea dinámica
    n_steps = trajectory.shape[0]
    # Declaramos una lista para almacenar los valores de MSD
    msd_values = []

    # Recorremos la trayectoria desde el segundo paso hasta el último
    for i in range(1, n_steps):
        # Inicializamos la distancia total para cada paso i
        total_distance = 0
        # Recorremos los pasos de la trayectoria desde el inicio hasta n_steps - i
        for j in range(n_steps - i):
            # Calculamos la distancia entre los puntos j y j + i en la trayectoria
            total_distance += distancia(trajectory.x_pos[j], trajectory.y_pos[j], trajectory.x_pos[j + i], trajectory.y_pos[j + i])

        # Calculamos el valor de MSD para el paso i y guardarlo en la lista
        msd_values.append((total_distance / (n_steps - i))**2)

    # Creamos un DataFrame de pandas con las columnas 'x_pos', 'y_pos' y 'z_pos'
    # 'z_pos' contiene los valores de MSD calculados
    msd_df = pd.DataFrame({'x_pos': trajectory.x_pos[:-1], 'y_pos': trajectory.y_pos[:-1], 'z_pos': msd_values})

    return msd_df

In [124]:
# Calcula el MSD para cada trayectoria
MSD_BM = funcion_msd(BM_2d_df_6)
MSD_CRW = funcion_msd(CRW_2d_df_9)

In [125]:
# Init figure
fig_MSD = go.Figure()

# add BM6
fig_MSD.add_trace(go.Scatter(
    x=MSD_BM.x_pos.index,
    y=MSD_BM.z_pos,
    name="MSD BM6",
    showlegend=True
))

fig_MSD.add_trace(go.Scatter(
    x=MSD_CRW.x_pos.index,
    y=MSD_CRW.z_pos,
    name='MSD CRW 6, c = 0.9',
    showlegend=True
))
fig_MSD.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [14]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [1]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    #Declaramos un vector que va desde la pos_a hasta la pos_b
    vec_ab = np.array([pos_b[0] - pos_a[0], pos_b[1] - pos_a[1]])
    #Se calcula la longitud de ese vector
    norm_ab = np.linalg.norm(vec_ab)

    #Declaramos un vector que va desde la pos_b hasta la pos_c
    vec_bc = np.array([pos_c[0] - pos_b[0], pos_c[1] - pos_b[1]])
    #Se calcula la longitud de ese vector
    norm_bc = np.linalg.norm(vec_bc)

    #Multiplicamos los vectores
    dot_p = np.dot(vec_ab, vec_bc)

    #Se calcula el coseno del ángulo entre los vectores
    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab * norm_bc + np.finfo(float).eps)

    # Angle orientation
    #Calculamos el producto de cruz o producto vectorial
    cross_p = np.cross(vec_ab, vec_bc)
    #Determinamos la orientación del ángulo
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    #La función arccos de NumPy devuelve el arcocoseno de los elementos de la estructura de entrada
    #Calculamos el ángulo en radianes utilizando arccos y se multiplica por la orientación
    theta = np.arccos(np.around(cos_theta,4)) * orient

    return theta

In [15]:
# aux to store turning angles
def auxiliar_angulos(trajectory):
  aux_ta_CRW_2d_df_6 = np.empty(shape=(0))

  # Iterate over trajectory compute turning angles
  for i, row in CRW_2d_df_6[1:-1].iterrows():
    # Calculamos los ángulos utilizando la función turning_angle, enviando las 3 posiciones
    angle = turning_angle(trajectory.iloc[i-1],trajectory.iloc[i],trajectory.iloc[i+1])
    #temp_df = pd.DataFrame([{'x_pos':df.x_pos[i],'y_pos':df.y_pos[i],'z_pos':angle}])
    aux_ta_CRW_2d_df_6  = np.append(aux_ta_CRW_2d_df_6 , angle)
    TA_df = pd.DataFrame(aux_ta_CRW_2d_df_6, columns=['TA'])
  return TA_df



In [16]:
def calculate_turning_angles(df):
    # DataFrame para almacenar los ángulos de giro
    turning_angles_df = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_pos'])

    # Iterar sobre la trayectoria para calcular los ángulos de giro
    for i in range(1, len(df) - 1):
        pos_a = df.iloc[i - 1].to_numpy()
        pos_b = df.iloc[i].to_numpy()
        pos_c = df.iloc[i + 1].to_numpy()

        # Calcular el ángulo de giro utilizando la función turning_angle
        angle = turning_angle(pos_a, pos_b, pos_c)

        # Crear un DataFrame temporal con las posiciones y el ángulo
        temp_df = pd.DataFrame([{'x_pos': df.x_pos[i], 'y_pos': df.y_pos[i], 'z_pos': angle}])

        # Concatenar el DataFrame temporal al DataFrame principal
        turning_angles_df = pd.concat([turning_angles_df, temp_df], ignore_index=True)

    return turning_angles_df


In [17]:
# Get Turning Angles calling the function
#ta_CRW_2d_df_6 = # Add your code here function(CRW_2d_df_6)
ta_CRW_2d_df_6 = calculate_turning_angles(CRW_2d_df_6)

# Get Turning Angles calling the function
#ta_CRW_2d_df_9 = # Add your code here function(CRW_2d_df_9)
ta_CRW_2d_df_9 = calculate_turning_angles(CRW_2d_df_9)

# Visualizar el resultado
print(ta_CRW_2d_df_6)
print(ta_CRW_2d_df_9)

          x_pos       y_pos     z_pos
0     -3.692764    6.895372  1.526081
1     -5.840528    1.292953 -1.995115
2    -10.061820    5.556835  1.248337
3    -15.443752    2.904514  0.478471
4    -19.000125   -1.927899 -0.547363
..          ...         ...       ...
993  590.010564  311.167948 -0.615127
994  594.061618  306.742006 -0.762372
995  593.934953  300.743343  0.024496
996  593.954299  294.743374 -0.637645
997  590.398341  289.910656 -0.157005

[998 rows x 3 columns]
       x_pos        y_pos     z_pos
0       7.99     4.660939  0.131243
1      13.97     5.108089 -0.116685
2      19.97     4.853790  0.311092
3      25.75     6.445185 -0.086050
4      31.65     7.534745  1.130398
..       ...          ...       ...
993  1023.29  1219.233664 -0.199832
994  1029.21  1220.233933 -0.877339
995  1033.76  1216.323266  0.140115
996  1038.81  1213.086230  0.044725
997  1044.00  1210.076997  0.619274

[998 rows x 3 columns]


In [134]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histograma
fig_met_df_3 = go.Figure()

# Definimos los límites del rango en el eje x
x_range = [-3, 3]

# Histogram turning angle CRW_2d_df_6
# Agregamos el histograma para los ángulos de giro de CRW_2d_df_6
fig_met_df_3.add_trace(go.Histogram(x=ta_CRW_2d_df_6.z_pos,
                                    name="Observada 0.6",
                                    xbins=dict(
                                    start=-3.0,
                                    end=4)))


# Histogram turning angle CRW_2d_df_9
# Agregamos el histograma para los ángulos de giro de CRW_2d_df_6
fig_met_df_3.add_trace(go.Histogram(x=ta_CRW_2d_df_9.z_pos,
                                    name="Observada 0.9",
                                    xbins=dict(
                                    start=-3.0,
                                    end=4)))


# Add origin distributions
#distribucion = wrapcauchy(0.6,loc=0,scale=1,)
#cauchy_pdf_6 = np.array([distribucion.pdf(i)for i in aux_domain_6])
#wrapcauchy_pdf_9 = np.array(wrapcauchy.pdf(aux_domain_6,*wrapcauchy_6))
cauchy_6 = cauchy.fit(ta_CRW_2d_df_6.z_pos)
aux_domain_6 = np.linspace(-3,3,1000)
cauchy_pdf_6 = cauchy.pdf(aux_domain_6,*cauchy_6)


fig_met_df_3.add_trace(go.Scatter(x=aux_domain_6,
                                        y=cauchy_pdf_6,
                                        mode='lines',
                                        name='cauchy 0.6',
                                        showlegend=True))

# Add origin distributions
# Init parameters
#distribucion = wrapcauchy(0.9,loc=0,scale=1,)
#wrapcauchy_pdf_9 = np.array(wrapcauchy.pdf(aux_domain_9,*wrapcauchy_9))
cauchy_9 = cauchy.fit(ta_CRW_2d_df_9.z_pos)
aux_domain_9 = np.linspace(-3,3,1000)
cauchy_pdf_9 = cauchy.pdf(aux_domain_9,*cauchy_9)


fig_met_df_3.add_trace(go.Scatter(x=aux_domain_9,
                                        y=cauchy_pdf_9,
                                        mode='lines',
                                        name='cauchy 0.9',
                                        showlegend=True))

fig_met_df_3.update_layout(xaxis=dict(range=x_range))

fig_met_df_3.show()

**Nota:** No logré proyectar la distribución origen vs distribución observada en la misma escala y se me terminó el tiempo, pero si se tienen el mismo comportamiento.

In [135]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histograma
fig_met_df_3 = go.Figure()

# Definimos los límites del rango en el eje x
x_range = [-3, 3]


# Add origin distributions
#distribucion = wrapcauchy(0.6,loc=0,scale=1,)
#cauchy_pdf_6 = np.array([distribucion.pdf(i)for i in aux_domain_6])
#wrapcauchy_pdf_9 = np.array(wrapcauchy.pdf(aux_domain_6,*wrapcauchy_6))
cauchy_6 = cauchy.fit(ta_CRW_2d_df_6.z_pos)
aux_domain_6 = np.linspace(-3,3,1000)
cauchy_pdf_6 = cauchy.pdf(aux_domain_6,*cauchy_6)


fig_met_df_3.add_trace(go.Scatter(x=aux_domain_6,
                                        y=cauchy_pdf_6,
                                        mode='lines',
                                        name='cauchy 0.6',
                                        showlegend=True))

# Add origin distributions
# Init parameters
#distribucion = wrapcauchy(0.9,loc=0,scale=1,)
#wrapcauchy_pdf_9 = np.array(wrapcauchy.pdf(aux_domain_9,*wrapcauchy_9))
cauchy_9 = cauchy.fit(ta_CRW_2d_df_9.z_pos)
aux_domain_9 = np.linspace(-3,3,1000)
cauchy_pdf_9 = cauchy.pdf(aux_domain_9,*cauchy_9)


fig_met_df_3.add_trace(go.Scatter(x=aux_domain_9,
                                        y=cauchy_pdf_9,
                                        mode='lines',
                                        name='cauchy 0.9',
                                        showlegend=True))

fig_met_df_3.update_layout(xaxis=dict(range=x_range), yaxis=dict(range=[0, 3.5]))

fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [105]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [62]:
# Función que genera Levy walk
def levy_flight(n_steps, speed = 5,  s_pos=[0,0], CRW_exponent = 0.5, alpha = 1.5, beta = 0, loc = 3.0):
  levy_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos':s_pos[1]}])
  levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

  velocity = Vec2d(speed,0)
  for i in range(n_steps -1):
    turn_angle = wrapcauchy.rvs(CRW_exponent)
    step_size = levy_stable.rvs(alpha=alpha, beta=beta, loc=loc)
    velocity = velocity.rotated(turn_angle)

    # Calculamos la nueva posición y se agrega al DataFrame de trayectoria
    temp_df = pd.DataFrame([{'x_pos':levy_df.x_pos[i]+(velocity.x*step_size), 'y_pos':levy_df.y_pos[i]+(velocity.y*step_size)}])
    levy_df = pd.concat([levy_df,temp_df], ignore_index=True)
  return levy_df

In [69]:
def calculate_step_length(trayectoria):
    trayectoria_con_giro = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_angulo'])
    temp_df = pd.DataFrame([{'x_pos': trayectoria.x_pos[0], 'y_pos': trayectoria.y_pos[0], 'z_angulo': 0}])
    trayectoria_con_giro = pd.concat([trayectoria_con_giro, temp_df], ignore_index=True)

    for i in range(1, trayectoria.shape[0]-1):
        angulo = turning_angle(trayectoria.iloc[i-1], trayectoria.iloc[i], trayectoria.iloc[i+1])
        temp_df = pd.DataFrame([{'x_pos': trayectoria.x_pos[i], 'y_pos': trayectoria.y_pos[i], 'z_angulo': angulo}])
        trayectoria_con_giro = pd.concat([trayectoria_con_giro, temp_df], ignore_index=True)

    trayectoria_con_giro_y_pasos = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_angulo', 'z_steps'])
    temp_df = pd.DataFrame([{'x_pos': trayectoria_con_giro.x_pos[0], 'y_pos': trayectoria_con_giro.y_pos[0], 'z_angulo': 0, 'z_steps': 1}])
    trayectoria_con_giro_y_pasos = pd.concat([trayectoria_con_giro_y_pasos, temp_df], ignore_index=True)

    x_arista = trayectoria_con_giro.x_pos[0]
    y_arista = trayectoria_con_giro.y_pos[0]
    pasos = 1

    for i in range(1, trayectoria_con_giro.shape[0]-1):
        if trayectoria_con_giro.z_angulo[i] == 0:
            pasos += 1
        else:
            temp_df = pd.DataFrame([{'x_pos': trayectoria_con_giro.x_pos[i], 'y_pos': trayectoria_con_giro.y_pos[i],
                                     'z_angulo': trayectoria_con_giro.z_angulo[i], 'z_steps': pasos}])
            trayectoria_con_giro_y_pasos = pd.concat([trayectoria_con_giro_y_pasos, temp_df], ignore_index=True)
            x_arista = trayectoria_con_giro.x_pos[i]
            y_arista = trayectoria_con_giro.y_pos[i]
            pasos = 1

    return trayectoria_con_giro_y_pasos

In [106]:
# Get Step lengths calling the function
sl_levy_2d_df_1_et=calculate_step_length(Levy_2d_df_1)
distribucion=levy_stable(alpha=1.0, beta=1.0, loc=3.0)
aux_domain_1 = np.linspace(0, 100, 1000)
levy_pdf_1 = np.array([distribucion.pdf(i)for i in aux_domain_1])


sl_levy_2d_df_7_et=calculate_step_length(Levy_2d_df_7)
distribucion=levy_stable(alpha=0.7, beta=1.0, loc=3.0)
aux_domain_7 = np.linspace(0, 100, 1000)
levy_pdf_7 = np.array([distribucion.pdf(i)for i in aux_domain_7])

In [112]:
# PLot histogram
#fig_met_df_4 = go.Figure()

# Definimos los límites del rango de los ejes x, y
x_range = [0,800]
y_range = [0.02]

layout = go.Layout(
    yaxis=dict(range=[0,0.02]),
    xaxis=dict(range=[0, 800])
)

# Histogram step-length Levy_2d_df_1
fig_met_df_4 = go.Figure(data=[go.Histogram(x=sl_levy_2d_df_1_et.z_steps,
                                   histnorm='probability density',
                                   name='Observed alpha=1.0 beta=1.0',
                                   )],
                                   layout=layout)


fig_met_df_4.add_histogram(x=sl_levy_2d_df_7_et.z_steps,
                                   histnorm='probability density',
                                   name='Observed alpha=0.7 beta=1.0',
                                   )


# Add origin distributions
fig_met_df_4.add_trace(go.Scatter(x=aux_domain_1,
                                  y=levy_pdf_1,
                                  mode='lines',
                                  name='Levy aplha 1.0',
                                  showlegend=True
))

fig_met_df_4.add_trace(go.Scatter(x=aux_domain_7,
                                  y=levy_pdf_7,
                                  mode='lines',
                                  name='Levy alpha 0.7',
                                  showlegend=True
))

fig_met_df_4.update_layout(
    xaxis=dict(range=x_range),
    title='Step-length Distribution - (Dist. origen vs Dist. observada)'
)


fig_met_df_4.show()

In [122]:
#Generar levy flights
Levy_2d_df_1_generada = levy_flight(n_steps=10000, speed = 6, alpha=1.0, beta=1.0, loc=3.0)
sl_levy_2d_df_1=calculate_step_length(Levy_2d_df_1_generada)
distribucion=levy_stable(alpha=1.0, beta=1.0, loc=3.0)
aux_domain_1 = np.linspace(0, 100, 1000)
levy_pdf_1 = np.array([distribucion.pdf(i)for i in aux_domain_1])

Levy_2d_df_7_generada = levy_flight(n_steps=10000, speed = 6, alpha=0.7, beta=1.0, loc=3.0)
sl_levy_2d_df_7=calculate_step_length(Levy_2d_df_7_generada)
distribucion=levy_stable(alpha=0.7, beta=1.0, loc=3.0)
aux_domain_7 = np.linspace(0, 100, 1000)
levy_pdf_7 = np.array([distribucion.pdf(i)for i in aux_domain_7])

In [121]:
# PLot histogram
#fig_met_df_4 = go.Figure()

# Definimos los límites del rango de los ejes x, y
x_range = [0,400]
y_range = [0.02]

layout = go.Layout(
    yaxis=dict(range=[0,0.02]),
    xaxis=dict(range=[0, 400])
)

# Histogram step-length Levy_2d_df_1
fig_met_df_4 = go.Figure(data=[go.Histogram(x=sl_levy_2d_df_1.z_steps,
                                   histnorm='probability density',
                                   name='Observed alpha=1.0 beta=1.0',
                                   )],
                                   layout=layout)


fig_met_df_4.add_histogram(x=sl_levy_2d_df_7.z_steps,
                                   histnorm='probability density',
                                   name='Observed alpha=0.7 beta=1.0',
                                   )


# Add origin distributions
fig_met_df_4.add_trace(go.Scatter(x=aux_domain_1,
                                  y=levy_pdf_1,
                                  mode='lines',
                                  name='Levy aplha 1.0',
                                  showlegend=True
))

fig_met_df_4.add_trace(go.Scatter(x=aux_domain_7,
                                  y=levy_pdf_7,
                                  mode='lines',
                                  name='Levy alpha 0.7',
                                  showlegend=True
))

fig_met_df_4.update_layout(
    xaxis=dict(range=x_range),
    title='Step-length Distribution - (Dist. origen vs Dist. observada)'
)


fig_met_df_4.show()